## 주식 가격 (동종사)

### 과거 데이터 일괄로 가져와서 저장

In [17]:
# 동종사 코드
stock_codes = [
    '004690',   # 삼천리
    '017390',   # 서울도시가스
    '034590',   # 인천도시가스
    '267290'    # 경동도시가스
]

# 과거 기간
start_date = '20101224' # 대성에너지 주가 데이터 시작일(2010-12-24)부터 데이터 가져옴
end_date = '20230903'   # 2023년 9월 3일까지

In [18]:
from pykrx import stock
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for stock_code in stock_codes:
    # 특정기간 특정 종목의 OHLCV
    ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)
    df_ohlcv = ohlcv.reset_index()
    # 날짜 형식 변환 => 문자열 yyyymmdd
    df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
    df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')
    print(f'주식 종목 {stock_code}, {df_ohlcv.shape[0]}건의 데이터 저장')
    for row in df_ohlcv.itertuples():
        sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                                values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
    conn.commit()

conn.close()

주식 종목 004690, 3129건의 데이터 저장
주식 종목 017390, 3129건의 데이터 저장
주식 종목 034590, 3129건의 데이터 저장
주식 종목 267290, 1555건의 데이터 저장
